### This is the base code for Multi-modal fusion. This particular code is applied on activity data collected using Xethrue, 24 GHz and 77 GHz Data. A total of 11 different activties were considered. This was published in 2020 and won  IEEE Radar conf Best student paper: 'Multi-Frequency RF sensor data adaptation for motion recogition using multi-modal Deep learning'

In [1]:
# Pandas and Numpy for data structures and util fucntions
import scipy as sp
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 600

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.optimizers import RMSprop,Adam,SGD
from keras.layers import Input, ZeroPadding2D,concatenate, Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, UpSampling2D, BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import CSVLogger
from keras.utils import np_utils
from keras import optimizers
from livelossplot.keras import PlotLossesCallback
import efficientnet.keras as efn
import h5py, glob
import numpy as np
import itertools
from cnn_utils import *
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(1)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Using TensorFlow backend.


In [2]:
print(tf.__version__)
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

2.2.0-rc3
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
import os
# Create a MirroredStrateg, If Multi-GPU available
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
tf.config.set_soft_device_placement(True)

In [5]:
import cv2
import glob
import numpy as np
np.random.seed(1)
from sklearn.model_selection import train_test_split

# Load Modality 1 Data

In [6]:
import cv2
import glob
import os
import numpy as np
from keras.utils import to_categorical
np.random.seed(1)
X_data = []
mypath='/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/'
dirlist = sorted(os.listdir(mypath))
files = sorted(glob.glob ('/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/*/*.png'))
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    image = cv2.resize(image, (128,128), interpolation = cv2.INTER_AREA)
    X_data.append (image)

print('X_data shape:', np.array(X_data).shape)

# Create pattern to find files for labeling
dir_arr=['/' + s + '/' for s in dirlist]

# label according to the folder name
labels = []
for i in files:
    res1 = [ele for ele in dir_arr if(ele in i)]
    res  = res1[0][1:-1]
    idx  = dirlist.index(res)
    labels.append(idx)



X=np.array(X_data)
np_labels=np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, np_labels, test_size=0.20, random_state=42)

NUM_CLASSES=11
X_train = X_train/255.
X_test = X_test/255.


Y_train = convert_to_one_hot(y_train, NUM_CLASSES).T
Y_test = convert_to_one_hot(y_test, NUM_CLASSES).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837114_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837135_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837157_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837181_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837204_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837254_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837276_Raw_0 (copy).png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837276_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837305_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_1573837370_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Away/04060010_15

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574627935_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574627955_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574627986_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574628005_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574628024_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574628042_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080029_1574628060_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080030_1574194651_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080030_1574194672_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080030_1574194692_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Bend/04080030_157419494

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100010_1573838564_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100010_1573838584_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100010_1573838603_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698031_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698067_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698089_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698108_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698127_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698146_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/04100014_1574698165_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Kneel/0410001

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574196750_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574196770_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574196790_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574196811_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459511_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459528_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459546_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459566_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459585_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_1574459602_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Limp/04170030_157445962

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117555_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117573_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117591_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117609_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117627_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117647_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574117666_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574629378_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574629397_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/04180029_1574629415_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/SStep/0418002

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838030_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838066_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838086_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838127_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838157_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838182_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838201_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090010_1573838334_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090014_1574696977_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090014_1574696998_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Sit/04090014_1574697018_Raw_0.png

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196407_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196432_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196452_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196472_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196492_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196510_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196530_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196549_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574196569_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_1574459307_Raw_0.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/77GHz/Toes/04160030_157445932

### All the class accross modality should have same number of data samples

In [7]:
from collections import Counter
#Counter(np.argmax(Y_train,-1))
Counter(np_labels)

Counter({0: 60,
         1: 59,
         2: 61,
         3: 62,
         4: 59,
         5: 60,
         6: 60,
         7: 57,
         8: 57,
         9: 58,
         10: 60})

# Load  modality 2 Data

In [8]:
import cv2
import glob
import os
import numpy as np
from keras.utils import to_categorical
np.random.seed(1)
X_data2 = []
mypath='/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/'
dirlist = sorted(os.listdir(mypath))
files = sorted(glob.glob ('/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/*/*.png'))
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    image = cv2.resize(image, (128,128), interpolation = cv2.INTER_AREA)
    X_data2.append (image)

print('X_data shape:', np.array(X_data).shape)

# Create pattern to find files for labeling
dir_arr=['/' + s + '/' for s in dirlist]

# label according to the folder name
labels2 = []
for i in files:
    res2 = [ele for ele in dir_arr if(ele in i)]
    res22  = res2[0][1:-1]
    idx2  = dirlist.index(res22)
    labels2.append(idx2)
    
X2=np.array(X_data2)
print(X2.shape)
np_labels22=np.array(labels2)
X_train10, X_test10, y_train10, y_test10 = train_test_split(X2, np_labels22, test_size=0.20, random_state=42)


X_train_2 = X_train10/255.
X_test_2 = X_test10/255.



Y_train_2 = convert_to_one_hot(y_train10, NUM_CLASSES).T
Y_test_2 = convert_to_one_hot(y_test10, NUM_CLASSES).T

print ("number of training examples = " + str(X_train_2.shape[0]))
print ("number of test examples = " + str(X_test_2.shape[0]))
print ("X_train_2 shape: " + str(X_train_2.shape))
print ("y_train_2 shape: " + str(y_train10.shape))
print ("X_test_2 shape: " + str(X_test_2.shape))
print ("Y_test_2 shape: " + str(Y_test_2.shape))

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837114.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837135.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837157.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837181.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837204.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837254.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837276.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837305.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060010_1573837370.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060014_1574696344.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Away/01060014_1574696370.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24G

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Limp/01160030_1574459399.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Limp/01160030_1574459417.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Limp/01160030_1574459436.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Limp/01160030_1574459454.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Limp/01160030_1574459473.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837435.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837456.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837478.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837502.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837531.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Pick/01070010_1573837550.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24G

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170010_1573840150.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170010_1573840168.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170010_1573840212.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698742.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698787.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698907.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698926.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698947.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698967.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574698987.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24GHz/Toes/01170014_1574699007.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/24G

number of training examples = 522
number of test examples = 131
X_train_2 shape: (522, 128, 128, 3)
y_train_2 shape: (522,)
X_test_2 shape: (131, 128, 128, 3)
Y_test_2 shape: (131, 11)


In [9]:
from collections import Counter
Counter(np.argmax(Y_train_2,-1))
#Counter(np_labels22)

Counter({10: 46,
         6: 50,
         0: 46,
         4: 47,
         3: 49,
         9: 49,
         8: 50,
         5: 47,
         7: 47,
         1: 41,
         2: 50})

## Load Modality 3 Data

In [10]:
import cv2
import glob
import os
import numpy as np
from keras.utils import to_categorical
np.random.seed(1)
X_data3 = []
mypath='/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/'
dirlist = sorted(os.listdir(mypath))
files = sorted(glob.glob ('/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/*/*.png'))
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    image = cv2.resize(image, (128,128), interpolation = cv2.INTER_AREA)
    X_data3.append (image)

print('X_data shape:', np.array(X_data3).shape)

# Create pattern to find files for labeling
dir_arr=['/' + s + '/' for s in dirlist]

# label according to the folder name
labels3 = []
for i in files:
    res3 = [ele for ele in dir_arr if(ele in i)]
    res33  = res3[0][1:-1]
    idx3  = dirlist.index(res33)
    labels3.append(idx3)

X3=np.array(X_data3)
print(X3.shape)
np_labels3=np.array(labels3)
X_train77, X_test77, y_train77, y_test77 = train_test_split(X3, np_labels3, test_size=0.20, random_state=42)


X_train_3 = X_train77/255.
X_test_3 = X_test77/255.



Y_train_3 = convert_to_one_hot(y_train77, NUM_CLASSES).T
Y_test_3 = convert_to_one_hot(y_test77, NUM_CLASSES).T

print ("number of training examples = " + str(X_train_3.shape[0]))
print ("number of test examples = " + str(X_test_3.shape[0]))
print ("X_train_3 shape: " + str(X_train_3.shape))
print ("y_train_3 shape: " + str(Y_train_3.shape))
print ("X_test_3 shape: " + str(X_test_3.shape))
print ("Y_test_3 shape: " + str(Y_test_3.shape))

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837114.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837135.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837157.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837181.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837204.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837254.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837276.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837305.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060010_1573837370.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060014_1574696344.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Away/06060014_1574696370.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multi

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194651.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194672.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194692.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194713.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194941.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194962.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574194982.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574195000.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574195019.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574195038.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Bend/06080030_1574195059.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multi

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116046.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116064.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116083.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116103.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116122 (copy).png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116122.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116141 (copy).png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574116141.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574628484 (copy).png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574628484.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Kneel/06100029_1574628503 (copy).png
/mnt

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574459585.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574459602.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574459620.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574459638.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574459656.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Limp/06170030_1574460109.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Pick/06070010_1573837435.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Pick/06070010_1573837456.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Pick/06070010_1573837478.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Pick/06070010_1573837502.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Pick/06070010_1573837531.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multi

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629453.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629474.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629493.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629512.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629533.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629552.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180029_1574629570.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180030_1574196837.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180030_1574196864.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180030_1574196884.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/SStep/06180030_1574196905.png
/mnt/HDD02/RadarConf/Multimodal_

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574115842 (copy).png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574115842.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628100.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628120.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628141.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628160.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628180.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628246.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628309.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628343.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Sit/06090029_1574628440.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimoda

/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574114606.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574114625.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626078.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626109.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626164.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626188.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626210.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626307.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626331.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626372.png
/mnt/HDD02/RadarConf/Multimodal_paper/Multimodal21/Xethru/Towards/06050029_1574626392.png
/mnt/HDD02

In [11]:
from collections import Counter
#Counter(np.argmax(Y_train_3,-1))
Counter(np_labels3)

Counter({0: 60,
         1: 59,
         2: 61,
         3: 62,
         4: 59,
         5: 60,
         6: 60,
         7: 57,
         8: 57,
         9: 58,
         10: 60})

## Combined Model

In [22]:
import keras
acc_hist = []
hist_hist = []
num_class=11

# 24 GHz Model

input_img=keras.layers.Input(shape=(128, 128, 3))
conv01_24 = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img) # 128x128x8
conv02_24 = Conv2D(8, (9, 9), activation='relu', padding='same')(input_img)
out01_24  = concatenate([conv01_24,conv02_24],axis=-1)
pool01_24 = MaxPooling2D(pool_size=(4, 4))(out01_24) #14 x 14 x 32, 64x64x8

conv1_24 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool01_24) # 128x128x8
conv2_24 = Conv2D(16, (9, 9), activation='relu', padding='same')(pool01_24)
out1_24  = concatenate([conv1_24,conv2_24],axis=-1)
pool1_24 = MaxPooling2D(pool_size=(2, 2))(out1_24) #14 x 14 x 32, 64x64x8

conv3_24 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1_24) # 64x64x16
conv4_24 = Conv2D(32, (9, 9), activation='relu', padding='same')(pool1_24)
out2_24  = concatenate([conv3_24,conv4_24],axis=-1)
pool2_24 = MaxPooling2D(pool_size=(2, 2))(out2_24)
flat_24 = Flatten()(pool2_24)

# 77 GHz Model
input_img_77=keras.layers.Input(shape=(128, 128, 3))
conv01_77 = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img_77) # 128x128x8
conv02_77 = Conv2D(8, (9, 9), activation='relu', padding='same')(input_img_77)
out01_77  = concatenate([conv01_77,conv02_77],axis=-1)
pool01_77 = MaxPooling2D(pool_size=(4, 4))(out01_77) #14 x 14 x 32, 64x64x8

conv1_77 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool01_77) # 128x128x8
conv2_77 = Conv2D(16, (9, 9), activation='relu', padding='same')(pool01_77)
out1_77  = concatenate([conv1_77,conv2_77],axis=-1)
pool1_77 = MaxPooling2D(pool_size=(2, 2))(out1_77) #14 x 14 x 32, 64x64x8

conv3_77 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1_77) # 64x64x16
conv4_77 = Conv2D(32, (9, 9), activation='relu', padding='same')(pool1_77)
out2_77  = concatenate([conv3_77,conv4_77],axis=-1)
pool2_77 = MaxPooling2D(pool_size=(2, 2))(out2_77)
flat_77 = Flatten()(pool2_77)


# 10 GHz Model
input_img_10=keras.layers.Input(shape=(128, 128, 3))
conv01_10 = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img_10) # 128x128x8
conv02_10 = Conv2D(8, (9, 9), activation='relu', padding='same')(input_img_10)
out01_10  = concatenate([conv01_10,conv02_10],axis=-1)
pool01_10 = MaxPooling2D(pool_size=(4, 4))(out01_10) #14 x 14 x 32, 64x64x8

conv1_10 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool01_10) # 128x128x8
conv2_10 = Conv2D(16, (9, 9), activation='relu', padding='same')(pool01_10)
out1_10  = concatenate([conv1_10,conv2_10],axis=-1)
pool1_10 = MaxPooling2D(pool_size=(2, 2))(out1_10) #14 x 14 x 32, 64x64x8

conv3_10 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1_10) # 64x64x16
conv4_10 = Conv2D(32, (9, 9), activation='relu', padding='same')(pool1_10)
out2_10  = concatenate([conv3_10,conv4_10],axis=-1)
pool2_10 = MaxPooling2D(pool_size=(2, 2))(out2_10)
flat_10 = Flatten()(pool2_10)


high1 = concatenate([flat_24, flat_77,flat_10]) 
# high2 = Dense(128, activation='relu')(high1)

# high3 = Dropout(0.55)(high2)
# high4 = Dense(128, activation='relu')(high3)
# high5 = Dropout(0.55)(high4)
high6 = Dense(num_class)(high1)
final_out = Activation('softmax')(high6)
model = Model(inputs=[input_img, input_img_77,input_img_10], outputs=final_out,  name="24+77GHz_model")
model.summary()

Model: "24+77GHz_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_73 (Conv2D)              (None, 128, 128, 8)  224         input_13[0][0]                   
_____________________________________________________________________________________

In [ ]:
# for layer in model.layers[-7:]:
#      layer.trainable = True
# for layer in model.layers:
#     print(layer, layer.trainable)


In [23]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.001, decay = 1e-06), metrics=['accuracy'])

history = model.fit([X_train, X_train_2,X_train_3],Y_train_2,
                batch_size=16,
                epochs=50,
                verbose=1,
                validation_data=([X_test,X_test_2,X_test_3], Y_test),                
                shuffle=False
#                 callbacks = [PlotLossesCallback()]
                     )
acc = history.history['val_accuracy'][-1]
acc_hist.append(acc)
hist_hist.append(history.history)  

Train on 522 samples, validate on 131 samples
Epoch 1/50
522/522 [==============================] - 2s 4ms/step - loss: 1.6094 - accuracy: 0.4368 - val_loss: 0.7167 - val_accuracy: 0.7405
Epoch 2/50
522/522 [==============================] - 1s 2ms/step - loss: 0.5864 - accuracy: 0.7969 - val_loss: 0.3666 - val_accuracy: 0.8779
Epoch 3/50
522/522 [==============================] - 1s 2ms/step - loss: 0.3498 - accuracy: 0.8870 - val_loss: 0.2778 - val_accuracy: 0.9084
Epoch 4/50
522/522 [==============================] - 1s 2ms/step - loss: 0.2429 - accuracy: 0.9157 - val_loss: 0.1914 - val_accuracy: 0.9313
Epoch 5/50
522/522 [==============================] - 1s 2ms/step - loss: 0.1428 - accuracy: 0.9464 - val_loss: 0.2711 - val_accuracy: 0.8855
Epoch 6/50
522/522 [==============================] - 1s 2ms/step - loss: 0.1344 - accuracy: 0.9540 - val_loss: 0.2098 - val_accuracy: 0.9084
Epoch 7/50
522/522 [==============================] - 1s 2ms/step - loss: 0.0893 - accuracy: 0.9770 - 

In [ ]:
preds = model.evaluate([X_test,X_test_2,X_test_3], Y_test_2)
print ('Classification of 20 words through feature level fusioning 77,24 & 10 GHz:')
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
from sklearn.metrics import confusion_matrix

tt= model.predict([X_test,X_test_2,X_test_3])
cm=confusion_matrix(np.argmax(Y_test_2,axis=1), np.argmax(tt,axis=1))
print(cm)

In [ ]:
import pandas as pd
import seaborn as sn
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 4}

plt.rc('font', **font)
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10)
Y_pred = np.argmax(tt, -1)
Y_test_int = np.argmax(Y_test, -1)
data = {'y_Predicted': Y_pred,
        'y_Actual':    Y_test_int
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], normalize='index',rownames=['Actual'], colnames=['Predicted'], margins = False)\
.round(3)*100
fig = plt.figure(dpi=150)
sn.heatmap(confusion_matrix,annot=True, cmap="Greens", cbar_kws={'format': '%.0f%%'}, fmt='g') 
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b,t)
cap = plt.gcf()
#plt.savefig('results/cfar+RD.png')
plt.show
plt.savefig('Feture level fusion confusion matrix.png')

In [ ]:
##### import pandas as pd
import seaborn as sn
font = {'family' : 'normal', 
        'weight' : 'bold',
        'size'   : 10}

plt.rc('font', **font) 
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12) 
Y_pred = np.argmax(pred9, -1)
Y_test_int = actuals2.astype(int)
data = {'y_Predicted': Y_pred,
        'y_Actual':    Y_test_int.astype(str)
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], normalize='index', rownames=['Actual'], 
                               colnames=['Predicted'], margins = False)\
.round(3)*100
fig = plt.figure(dpi=150)
sn.heatmap(confusion_matrix,annot=True, cmap="Greens", cbar_kws={'format': '%.0f%%'}, fmt='g') 
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b,t)
cap = plt.gcf()
#plt.savefig('results/cfar+RD.png')
plt.show